<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Magnetospheric Wald `GiRaFFEfood` Initial Data for `GiRaFFE`

## Author: Patrick Nelson

### NRPy+ Source Code for this module: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Magnetospheric_Wald.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Magnetospheric_Wald.py)

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This tutorial notebook has been confirmed to be self-consistent with its corresponding NRPy+ module, as documented [below](#code_validation1). The initial data has validated against the original `GiRaFFE`, as documented [here](Tutorial-Start_to_Finish_UnitTest-GiRaFFEfood_NRPy.ipynb).

## Introduction: 
With the `GiRaFFE` evolution thorn constructed, we now need to "feed" our giraffe with initial data to evolve. There are several different choices of initial data we can use here; here, we will only be implementing the "Magnetospheric Wald" initial data, given by Table 3 in [the original paper](https://arxiv.org/pdf/1704.00599.pdf):
\begin{align}
A_{i} &= \frac{C_0}{2} (g_{i \phi} + 2 a g_{t i}) \\
E_{i} &= 0 \\
\end{align}
(the unspecified components are set to 0). Here, $C_0$ is a constant that we will set to $1$ in our simulations. Now, to use this initial data scheme, we need to transform the above into the quantities actually tracked by `GiRaFFE` and HydroBase: $A_i$, $B^i$, $\tilde{S}_i$, $v^i$, and $\Phi$. Of these quantities, `GiRaFFEfood` will only set $A_i$, $v^i$, and $\Phi=0$, then call a separate function to calculate  $\tilde{S}_i$; `GiRaFFE` itself will call a function to set $B^i$ before the time-evolution begins. This can be done with eqs. 16 and 18, here given in that same order:
\begin{align}
v^i &= \alpha \frac{\epsilon^{ijk} E_j B_k}{B^2} -\beta^i \\
B^i &= \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k \\
\end{align}
In the simulations, $B^i$ will be calculated numerically from $A_i$; however, it will be useful to analytically calculate $B^i$ to use calculating the initial $v^i$.

<a id='toc'></a>

# Table of Contents:
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Import core NRPy+ modules and set NRPy+ parameters
1. [Step 2](#set_a_i): Set the vector $A_i$
1. [Step 3](#set_vi): Calculate $v^i$ from $A_i$ and $E_i$
1. [Step 4](#code_validation1): Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ Module
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Import core NRPy+ modules and set NRPy+ parameters \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

Here, we will import the NRPy+ core modules, set the reference metric to Cartesian, and set commonly used NRPy+ parameters. We will also set up a parameter to determine what initial data is set up, although it won't do much yet.



In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step 0.a: Import the NRPy+ core modules and set the reference metric to Cartesian
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Common_Functions as gfcf # Some useful functions for GiRaFFE initial data.

import reference_metric as rfm
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

# Step 1a: Set commonly used parameters.
thismodule = "GiRaFFEfood_NRPy_Magnetospheric_Wald"

C0 = par.Cparameters("REAL",thismodule,"C0",1.0)

<a id='set_a_i'></a>

# Step 2: Set the vector $A_i$ \[Back to [top](#toc)\]
$$\label{set_a_i}$$

We will first build the fundamental vectors $A_i$ and $E_i$ in spherical coordinates (see [Table 3](https://arxiv.org/pdf/1704.00599.pdf)). Note that we use reference_metric.py to set $r$ and $\theta$ in terms of Cartesian coordinates; this will save us a step later when we convert to Cartesian coordinates. Since $C_0 = 1$,
$$A_{i} = \frac{C_0}{2} (g_{i \phi} + 2 a g_{t i}).$$

In [2]:
# Step 2: Set the vectors A and E in Spherical coordinates
def Ar_MW(r,theta,phi, **params):
    g4DD = params["g4DD"]
    a    = params["a"]
    return sp.Rational(1,2) * C0 * (g4DD[1][3] + 2 * a * g4DD[0][1])

def Ath_MW(r,theta,phi, **params):
    g4DD = params["g4DD"]
    a    = params["a"]
    return sp.Rational(1,2) * C0 * (g4DD[2][3] + 2 * a * g4DD[0][2])

def Aph_MW(r,theta,phi, **params):
    g4DD = params["g4DD"]
    a    = params["a"]
    return sp.Rational(1,2) * C0 * (g4DD[3][3] + 2 * a * g4DD[0][3])

<a id='set_vi'></a>

# Step 3: Calculate $v^i$ from $A_i$ and $E_i$ \[Back to [top](#toc)\]
$$\label{set_vi}$$

Next, we will code up the Valencia velocity, which will require us to first code the electric and magnetic fields. 

The electric field is
$$E_{i} = 0$$\
Since $$v_{(n)}^i = \frac{\epsilon^{ijk} E_j B_k}{B^2},$$ we trivially find $$v_{(n)}^i = 0,$$

In [3]:
#Step 3: Compute v^i from A_i and E_i
def ValenciavU_func_MW(**params):
    return ixp.zerorank1()

<a id='code_validation1'></a>

# Step 4: Code Validation against `GiRaFFEfood_NRPy.GiRaFFEfood_NRPy` NRPy+ module  \[Back to [top](#toc)\]
$$\label{code_validation1}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the `GiRaFFE` Exact Wald initial data equations  we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFEfood_NRPy.GiRaFFEfood_NRPy_Exact_Wald](../edit/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_Exact_Wald.py) module.


In [4]:
import GiRaFFEfood_NRPy.GiRaFFEfood_NRPy as gf
import BSSN.ShiftedKerrSchild as sks
sks.ShiftedKerrSchild(True)
import reference_metric as rfm   # NRPy+: Reference metric support
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()

import BSSN.ADMBSSN_tofrom_4metric as AB4m
AB4m.g4DD_ito_BSSN_or_ADM("ADM",sks.gammaSphDD,sks.betaSphU,sks.alphaSph)
g4DD = AB4m.g4DD

for mu in range(4):
    for nu in range(4):
        g4DD[mu][nu] = g4DD[mu][nu].subs(sks.r,rfm.xxSph[0]).subs(sks.th,rfm.xxSph[1])

A_mwD = gfcf.Axyz_func_spherical(Ar_MW,Ath_MW,Aph_MW,stagger_enable = True,a=sks.a,g4DD = g4DD)
Valenciav_mwD = ValenciavU_func_MW()
gf.GiRaFFEfood_NRPy_generate_initial_data(ID_type = "MagnetosphericWald", stagger_enable = True,a=sks.a,g4DD = g4DD)

print("Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:")

def consistency_check(quantity1,quantity2,string):
    if quantity1-quantity2==0:
        print(string+" is in agreement!")
    else:
        print(string+" does not agree!")
        sys.exit(1)

for i in range(3):
    consistency_check(Valenciav_mwD[i],gf.ValenciavU[i],"ValenciavU"+str(i))
    consistency_check(A_mwD[i],gf.AD[i],"AD"+str(i))

Consistency check between GiRaFFEfood_NRPy tutorial and NRPy+ module:
ValenciavU0 is in agreement!
AD0 is in agreement!
ValenciavU1 is in agreement!
AD1 is in agreement!
ValenciavU2 is in agreement!
AD2 is in agreement!


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFEfood_NRPy.pdf](Tutorial-GiRaFFEfood_NRPy.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [5]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GiRaFFEfood_NRPy-Magnetospheric_Wald",location_of_template_file=os.path.join(".."))

Created Tutorial-GiRaFFEfood_NRPy-Magnetospheric_Wald.tex, and compiled
    LaTeX file to PDF file Tutorial-GiRaFFEfood_NRPy-
    Magnetospheric_Wald.pdf
